In [2]:
!pip install spacy
!pip install langdetect
!pip install emoji==1.7
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 34.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 42.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2

[notice] A new release of pip is available: 2

In [ ]:
'''!pip install spacy
!pip install langdetect
!pip install emoji==1.7
!python -m spacy download en_core_web_lg'''
import os
import re
import pandas as pd
import glob
from langdetect import detect
import csv
import sys
import emoji
import spacy
import regex
import re
import string
from collections import Counter
import gensim
from gensim import corpora, models
# Calculate coherence score
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

In [59]:
def load_dataset(path):
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    df_list = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file, header=None)
        df['COMPANY'] = csv_file.split("/")[-1].split('layoffs_scraped_tweets.csv')[0].capitalize()
        df_list.append(df)
    main_df = pd.concat(df_list, axis=0)
    main_df.columns = ['USER', 'TWEET', 'TIMESTAMP', 'LINK', 'COMPANY']
    return main_df

In [60]:
df = load_dataset('Tweets_Data/')
df.shape

(87872, 5)

In [61]:
# function to detect the language of a sentence
def detect_language(text):
    try:
        lang = detect(text)
    except:
        lang = 'unknown'
    return lang

# apply the detect_language function to the TEXT column and create a new column
df['LANGUAGE'] = df['TWEET'].apply(detect_language)

# remove rows where the LANGUAGE column is not 'en' (English)
df = df[df['LANGUAGE'] == 'en'].drop(columns=['LANGUAGE'])

In [62]:
df.to_csv('final_data.csv', index=False)

In [63]:
df = pd.read_csv('final_data.csv')

In [64]:
df.head()

,USER,TWEET,TIMESTAMP,LINK,COMPANY
0,franchisefun,What If Franchise Business Ownership Is Not Yo...,2023-04-13 12:40:17+00:00,https://twitter.com/franchisefun/status/164649...,Twitter
1,franchisefun,STOP! You Want To Start A Business With No Pri...,2023-04-12 14:38:27+00:00,https://twitter.com/franchisefun/status/164616...,Twitter
2,jais_george,"Musk says Twitter is roughly breaking even, ha...",2023-04-12 10:00:09+00:00,https://twitter.com/jais_george/status/1646090...,Twitter
3,ETTelecom,"Musk says Twitter is roughly breaking even, ha...",2023-04-12 10:00:07+00:00,https://twitter.com/ETTelecom/status/164609081...,Twitter
4,franchisefun,We Don't Know What We Don't Know!\n\nHow Can Y...,2023-04-11 14:23:11+00:00,https://twitter.com/franchisefun/status/164579...,Twitter


In [65]:
# Define a function to replace emojis with their descriptions
def replace_emojis(text):
    for emoji_char in emoji.emojize(text).split():
        description = emoji.demojize(emoji_char).replace(":","")
        text = text.replace(emoji_char, description)
    return text

def data_cleaning(main_df):
    
    
    # dropping duplicates from the dataframe
    main_df.drop_duplicates(inplace = True) 

    # define the regular expression pattern to match @ links
    at_pattern = re.compile(r'@\S+')
    # remove http links from the TWEET column
    
    main_df['TWEET'].replace(at_pattern, '', regex=True, inplace = True)

    # define the regular expression pattern to match http links
    http_pattern = re.compile(r'http\S+')
    # remove http links from the TWEET column
    main_df['TWEET'].replace(http_pattern, '', regex=True, inplace = True)

    # define the regular expression pattern to match words with hashtags
    hashtag_word_pattern = re.compile(r'(\#\w+)')
    # extract words with hashtags from the TWEET column and place them in a new column
    main_df['HASHTAGS'] = main_df['TWEET'].str.extractall(hashtag_word_pattern)[0].groupby(level=0).apply(list)

    # remove words with hashtags from the TWEET column
    main_df['TWEET'].replace(hashtag_word_pattern, '', regex=True, inplace = True)

    # Replace emojis with their descriptions
    main_df['TWEET'] = main_df['TWEET'].apply(lambda x: " ".join(replace_emojis(sentence) for sentence in x.split()))
    
    return main_df


In [66]:
df = data_cleaning(df)
df.shape

(76179, 6)

In [67]:
df.dropna(subset=['TWEET'], inplace=True)
df.shape

(76179, 6)

In [68]:
df.to_csv('cleaned_data.csv', index=False)

In [69]:
df = pd.read_csv('cleaned_data.csv')

In [74]:
#  Preprocessing
import nltk

def preprocess_data(df, remove_stop, lemmetize,input_col_name, output_col_name):
    
    tokenizer = nltk.tokenize.TweetTokenizer(strip_handles=False, reduce_len=True)
    df[output_col_name] = df[input_col_name].apply(lambda x: tokenizer.tokenize(x.lower()) if isinstance(x,str) else [])
    
    if remove_stop:
        stop_words = set(nltk.corpus.stopwords.words('english'))
        df[output_col_name] = df[output_col_name].apply(lambda x: [w for w in x if w not in stop_words])
    
    df[output_col_name] = df[output_col_name].apply(lambda tokens: [token for token in tokens if len(token) > 1 and token.isalnum()])
    
    # code for lemmetization
    if lemmetize:
        pass
    
    df[output_col_name] = df[output_col_name].apply(lambda tokens: list(set(tokens)))

In [75]:
preprocess_data(df,True,True,"TWEET","preprocessed_TWEETS")

In [76]:
df["preprocessed_TWEETS"].head()

0    [dot, connecting, another, key, franchise, goa...
1    [stop, experience, prior, want, mad, business,...
2    [employees, even, twitter, musk, says, breakin...
3    [employees, even, twitter, musk, says, breakin...
4    [need, search, franchise, business, help, know...
Name: preprocessed_TWEETS, dtype: object

In [77]:
# create 

id2word = corpora.Dictionary(df["preprocessed_TWEETS"])

id2word.filter_extremes(no_below=2, no_above=.99)

corpus = [id2word.doc2bow(d) for d in df["preprocessed_TWEETS"]]


In [78]:
# LDA model analysis
def LDA(num_topics, workers,passes, num_of_words, id2word_dict, corpus):
    np.random.seed(42)
    lda = models.ldamulticore.LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word_dict,workers=workers, passes = passes)
    return lda,lda.print_topics(num_words=10)

In [ ]:
lda,topics = LDA(5,7,20,10,id2word,corpus) 
# Parameters : dataframe column, number of topics, workers for parallelization, passes (epochs ), filter_extremes, number of words per topic 

for topic_id, topic in topics:
    print('Topic {}: {}'.format(topic_id, topic))
    
print ("\n\n")

# Caluculate coherance score
coherence_model_lda = CoherenceModel(model=lda, texts=df["preprocessed_TWEETS"], dictionary=id2word, coherence='c_v')
print('Coherence Score:', coherence_model_lda.get_coherence())

In [149]:
from gensim.models import HdpModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import CountVectorizer

def HDP(df_column,num_topics,num_words):
    # Create a CountVectorizer object
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

    # Vectorize the data
    X = vectorizer.fit_transform([' '.join(tokens) for tokens in df_column])

    # Create a Gensim Dictionary object
    gensim_dict = Dictionary([vectorizer.get_feature_names_out()])

    # Convert the document-term matrix into a Gensim Corpus object
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

    # Train the HDP model
    hdp = HdpModel(corpus=corpus, id2word=gensim_dict)
    
    return hdp, hdp.print_topics(num_topics=num_topics, num_words=num_words),corpus

In [ ]:
hdp,topics,corpus = HDP(df["preprocessed_TWEETS"],5,10)

for topic in topics:
    print(topic)

# Caluculate coherance score
coherence_model_hdp = CoherenceModel(model=hdp, texts=df["preprocessed_TWEETS"], dictionary=id2word, coherence='c_v')
print('Coherence Score:', coherence_model_hdp.get_coherence())

In [225]:
# NMF analysis 
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer;
from sklearn.decomposition import NMF
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import normalize;

def NMF_(df_column, components):
    vectorizer = CountVectorizer(analyzer='word', max_features=10000, stop_words='english', lowercase=True)
    x_counts = vectorizer.fit_transform([' '.join(tokens) for tokens in df_column])
    transformer = TfidfTransformer(smooth_idf=False)
    x_tfidf = transformer.fit_transform(x_counts)
    xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
    model = NMF(n_components=components, init='nndsvd');
    model.fit(xtfidf_norm)

    return model, vectorizer

In [226]:
nmf,vectorizer=  NMF_(df["preprocessed_TWEETS"],10)

In [227]:
topic_word_matrix = nmf.components_

# Calculate the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(topic_word_matrix)

# Calculate the coherence score
coherence_score = np.sum(np.triu(cosine_sim_matrix, k=1)) / (topic_word_matrix.shape[0] * (topic_word_matrix.shape[0] - 1) / 2)
print(coherence_score)

perplexity = np.exp(-nmf.reconstruction_err_ / tfidf.shape[0])
print(perplexity)

0.031449499299053524
0.9985598333310292


In [222]:
def get_nmf_topics(model, n_top_words,vectorizer):
    
    feat_names = vectorizer.get_feature_names_out()
    
    word_dict = {};
    for i in range(num_topics):
        
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [223]:
num_topics = 10
nmf_df = get_nmf_topics(nmf, 5,vectorizer)
nmf_df

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,vacancies,post,twitter,ghana,hiring,loudspeaker,job,elon,apply,employees
1,link,star,fired,notifications,click,calling,dream,musk,jobs,layoffs
2,click,rocket,got,accra,houston,callingcollision,new,like,lagos,laid
3,apply,online,today,benefit,apply,looking,help,layoffs,location,fired
4,job,resume,new,miss,2023,level,search,fires,click,meta
5,directly,apply,team,posts,visit,great,fed,staff,limited,amazon
6,description,pakistan,trending,remember,chicago,built,looking,changed,manager,google
7,visit,lahore,going,share,york,seeking,pakistan,button,officer,like
8,bio,karachi,work,manager,los,collision,career,right,2023,company
9,houston,islamabad,staff,engineer,angeles,dot,postings,going,nigeria,people
